# Parallel computing and GPU programming with Julia 
## Part III: GPU programming
Alexis Montoison

In [1]:
using BenchmarkTools
using CUDA

Julia has first-class support for GPU programming through the following packages that target GPUs from major vendors:
- [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl) for NVIDIA GPUs
- [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl) for AMD GPUs
- [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) for Intel GPUs
- [Metal.jl](https://github.com/JuliaGPU/Metal.jl) for Apple M-series GPUs

CUDA.jl is the most mature and we will use it for the workshop.
AMDGPU.jl is somewhat behind but still ready for general use, while oneAPI.jl and Metal.jl are functional but might contain bugs, miss some features and provide suboptimal performance.

What is the difference between a CPU and a GPU? 

<img src='./Graphics/cpu_vs_gpu.png' width='700'>

<img src='./Graphics/meme_gpu.jpg' width='300'>

Some key aspects of GPUs that need to be kept in mind:
- The large number of compute elements on a GPU (in the thousands) can enable extreme scaling for data parallel tasks.
- GPUs have their own memory. This means that data needs to be transfered to and from the GPU during the execution of a program.
- Cores in a GPU are arranged into a particular structure. At the highest level they are divided into “streaming multiprocessors” (SMs). Some of these details are important when writing own GPU kernels.

![gpu](./Graphics/gpu.png)

GPU programming with Julia can be as simple as using a different array type instead of regular `Base.Array` arrays:
- `CuArray` from CUDA.jl for NVIDIA GPUs
- `ROCArray` from AMDGPU.jl for AMD GPUs
- `oneArray` from oneAPI.jl for Intel GPUs
- `MtlArray` from Metal.jl for Apple GPUs

These array types are subtypes of `GPUArrays` from [GPUArrays.jl](https://github.com/JuliaGPU/GPUArrays.jl) and closely resemble `Base.Array` which enables us to write generic code which works on both CPU and GPU arrays.

In [4]:
if CUDA.functional()
    A_d = CuArray([1,2,3,4])
    A_d .+= 1
end

4-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 2
 3
 4
 5

We can do the same operation with other subtypes of `GPUArrays`:
```julia
if AMDGPU.functional()
    A_d = ROCArray([1,2,3,4])
    A_d .+= 1
end

if oneAPI.functional()
    A_d = oneArray([1,2,3,4])
    A_d .+= 1
end

A_d = MtlArray([1,2,3,4])
A_d .+= 1
```

Moving an array back from the GPU to the CPU is simple:

In [5]:
if CUDA.functional()
    A = Array(A_d)
end

4-element Vector{Int64}:
 2
 3
 4
 5

However, the overhead of copying data to the GPU makes such simple calculations very slow.

Let’s have a look at a more realistic example: matrix multiplication.
We create two random arrays, one on the CPU and one on the GPU, and compare the performance:

In [8]:
if CUDA.functional()
    A = rand(2^12, 2^12)
    A_d = CuArray(A)

    @btime $A * $A
    CUDA.@time A_d * A_d
end

  573.211 ms (2 allocations: 128.00 MiB)


  0.666150 seconds (148 CPU allocations: 7.852 KiB) (1 GPU allocation: 128.000 MiB, 0.00% memmgmt time)


4096×4096 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 1032.67  1031.68  1028.22  1035.34  1034.16  …  1029.3   1030.93   1034.47
 1002.6   1021.2   1010.54  1015.79  1015.36     1011.56  1001.06   1003.26
 1025.8   1035.46  1019.12  1029.53  1022.36     1026.57  1019.43   1008.84
 1023.68  1032.35  1017.77  1020.74  1028.01     1018.98  1027.65   1016.25
 1020.99  1029.74  1007.86  1012.55  1015.62     1021.65  1013.08   1013.71
 1032.27  1041.9   1023.99  1040.96  1028.83  …  1040.42  1038.84   1030.26
 1024.15  1037.4   1010.27  1026.66  1026.66     1027.99  1034.4    1026.1
 1013.42  1016.75  1003.63  1014.28  1005.89     1023.0    995.707  1008.09
 1026.13  1028.38  1018.79  1029.14  1021.47     1025.18  1022.01   1014.15
 1026.46  1025.74  1007.86  1024.72  1020.56     1019.32  1014.24   1010.99
    ⋮                                         ⋱                        ⋮
 1018.75  1018.06  1004.0   1020.67  1030.53     1026.18  1019.75   1013.39
 1025.39  1028.22  1016.3   1030.17  1

In [9]:
if CUDA.functional()
    A = rand(Float32, 2^12, 2^12)
    A_d = CuArray(A)
    @btime $A * $A
    CUDA.@time A_d * A_d
end

  273.121 ms (2 allocations: 64.00 MiB)


  0.233722 seconds (435.95 k CPU allocations: 22.972 MiB) (1 GPU allocation: 64.000 MiB, 0.01% memmgmt time)


4096×4096 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1045.63  1054.6   1026.77   1045.75  1042.25  …  1030.96   1059.39  1044.02
 1024.28  1032.75   997.311  1024.63  1013.51     1017.36   1019.15  1017.23
 1034.15  1038.25  1007.79   1034.34  1022.76     1023.42   1037.81  1022.22
 1030.53  1027.09   995.217  1011.15  1004.87     1003.04   1027.02  1006.19
 1033.95  1041.93  1010.0    1039.49  1028.18     1028.47   1048.55  1025.83
 1051.29  1053.68  1025.8    1041.63  1036.12  …  1044.43   1054.34  1033.68
 1036.35  1038.97  1003.96   1031.6   1004.47     1022.85   1040.2   1021.7
 1016.27  1038.13   998.653  1006.87  1000.37     1004.78   1037.67  1010.55
 1041.17  1058.92  1017.04   1045.88  1026.89     1039.81   1045.81  1035.02
 1034.44  1046.93  1000.6    1016.45  1008.62     1016.7    1024.59  1017.94
    ⋮                                          ⋱                        ⋮
 1036.96  1044.72  1012.24   1039.13  1021.15     1031.71   1051.52  1030.16
 1049.45  1046.66  1024.33

GPUs normally perform significantly better for 32-bit floats. Some GPUs doesn't support 64-bit floats!

Many array operations in Julia are implemented using loops, processing one element at a time. Doing so with GPU arrays is very ineffective, as the loop won't actually execute on the GPU, but transfer one element at a time and process it on the CPU. As this wrecks performance, you will be warned when performing this kind of iteration:

In [ ]:
if CUDA.functional()
    A_d[1] = 3.0
end

Scalar indexing is only allowed in an interactive session, e.g. the REPL, because it is convenient when porting CPU code to the GPU. If you want to disallow scalar indexing, e.g. to verify that your application executes correctly on the GPU, call the allowscalar function:

In [ ]:
if CUDA.functional()
    CUDA.allowscalar(false)
    A_d[1] = 3.0
end

In a non-interactive session, e.g. when running code from a script or application, scalar indexing is disallowed by default. There is no global toggle to allow scalar indexing; if you really need it, you can mark expressions using allowscalar with do-block syntax or `@allowscalar` macro:

In [ ]:
if CUDA.functional()
    CUDA.allowscalar(false)

    CUDA.allowscalar() do
        A_d[1] += 1
    end

    CUDA.@allowscalar A_d[1] += 1
end

Nvidia provides CUDA toolkit, a collection of libraries that contain precompiled kernels for common operations like matrix multiplication ([cuBLAS](https://docs.nvidia.com/cuda/cublas/)), fast Fourier transforms ([cuFFT](https://docs.nvidia.com/cuda/cufft/)), linear solvers ([cuSOLVER](https://docs.nvidia.com/cuda/cusolver/)), sparse linear algebra ([CUSPARSE](https://docs.nvidia.com/cuda/cusparse/)), etc.
These kernels are wrapped in CUDA.jl and can be used directly with CuArrays.

The recommended way to use CUDA.jl is to let it automatically download an appropriate CUDA toolkit. CUDA.jl will check your driver's capabilities, which versions of CUDA are available for your platform, and automatically download an appropriate artifact containing all the libraries that CUDA.jl supports.

```julia
CUDA.set_runtime_version!(v"11.8")
```
To use a local installation, you can invoke the same API but set the version to `"local"`:
```julia
CUDA.set_runtime_version!("local")
```

In [13]:
if CUDA.functional()
    CUDA.versioninfo()
end

CUDA runtime 11.8, artifact installation
CUDA driver 12.1
NVIDIA driver 530.30.2

Libraries: 
- CUBLAS: 11.11.3
- CURAND: 10.3.0
- CUFFT: 10.9.0
- CUSOLVER: 

11.4.1
- CUSPARSE: 11.7.5
- CUPTI: 18.0.0
- NVML: 12.0.0+530.30.2

Toolchain:
- Julia: 1.8.5
- LLVM: 13.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2
- Device capability support: sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: NVIDIA GeForce RTX 2070 Super (sm_75, 5.998 GiB / 8.000 GiB available)


Let's do a guided tour of what is inside CUDA.jl!

In [14]:
if CUDA.functional()
    using CUDA.CUBLAS
    using CUDA.CUFFT
    using CUDA.CUSOLVER
    using CUDA.CUSPARSE
end

A powerful way to program GPUs with arrays is through Julia’s higher-order array abstractions.
The simple element-wise addition we saw above, `a .+= 1`, is an example of this, but more general constructs can be created with `broadcast`, `map`, `reduce`, `accumulate` etc:

In [15]:
if CUDA.functional()
    broadcast(-, A_d, 1)
end

4096×4096 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.723568   -0.543832   -0.0437073  …  -0.843247    -0.958513   -0.193473
 -0.534164   -0.0107928  -0.375073      -0.939359    -0.146964   -0.223677
 -0.0538156  -0.5883     -0.441566      -0.477366    -0.978755   -0.283492
 -0.759323   -0.765987   -0.3644        -0.75736     -0.466245   -0.347597
 -0.833889   -0.794054   -0.250753      -0.666238    -0.779951   -0.0234583
 -0.157873   -0.714707   -0.823822   …  -0.716101    -0.0476163  -0.78971
 -0.808679   -0.452048   -0.711343      -0.817588    -0.456819   -0.49476
 -0.922374   -0.980734   -0.6057        -0.0680697   -0.372029   -0.211417
 -0.19559    -0.430557   -0.947154      -0.210618    -0.853541   -0.396716
 -0.307655   -0.0204104  -0.0700517     -0.574448    -0.96172    -0.187183
  ⋮                                  ⋱                            ⋮
 -0.952737   -0.999112   -0.352112      -0.642825    -0.422917   -0.956365
 -0.097131   -0.10814    -0.185954      -0.861275    -

In [16]:
if CUDA.functional()
    map(x -> x+1, A_d)
end

4096×4096 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.27643  1.45617  1.95629  1.37037  …  1.0502   1.15675  1.04149  1.80653
 1.46584  1.98921  1.62493  1.75959     1.49312  1.06064  1.85304  1.77632
 1.94618  1.4117   1.55843  1.44545     1.54681  1.52263  1.02124  1.71651
 1.24068  1.23401  1.6356   1.77889     1.33846  1.24264  1.53375  1.6524
 1.16611  1.20595  1.74925  1.7153      1.01942  1.33376  1.22005  1.97654
 1.84213  1.28529  1.17618  1.74993  …  1.01643  1.2839   1.95238  1.21029
 1.19132  1.54795  1.28866  1.0058      1.61055  1.18241  1.54318  1.50524
 1.07763  1.01927  1.3943   1.03687     1.29877  1.93193  1.62797  1.78858
 1.80441  1.56944  1.05285  1.87457     1.23308  1.78938  1.14646  1.60328
 1.69235  1.97959  1.92995  1.92466     1.56531  1.42555  1.03828  1.81282
 ⋮                                   ⋱                             ⋮
 1.04726  1.00089  1.64789  1.82304     1.55603  1.35717  1.57708  1.04364
 1.90287  1.89186  1.81405  1.43621     1.85639  1.13

In [17]:
if CUDA.functional()
    reduce(+, A_d)
end

8.3885985f6

In [18]:
if CUDA.functional()
    accumulate(+, A_d)
end

4096×4096 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
    0.276432  2065.51  4137.75  6143.16  …  8.38244f6  8.38448f6  8.38656f6
    0.742269  2066.5   4138.37  6143.92     8.38244f6  8.38448f6  8.38656f6
    1.68845   2066.91  4138.93  6144.36     8.38244f6  8.38448f6  8.38656f6
    1.92913   2067.14  4139.57  6145.14     8.38244f6  8.38448f6  8.38656f6
    2.09524   2067.35  4140.32  6145.86     8.38244f6  8.38448f6  8.38656f6
    2.93737   2067.63  4140.49  6146.61  …  8.38244f6  8.38448f6  8.38656f6
    3.12869   2068.18  4140.78  6146.61     8.38244f6  8.38448f6  8.38656f6
    3.20632   2068.2   4141.18  6146.65     8.38244f6  8.38448f6  8.38656f6
    4.01073   2068.77  4141.23  6147.53     8.38244f6  8.38448f6  8.38656f6
    4.70307   2069.75  4142.16  6148.45     8.38244f6  8.38448f6  8.38656f6
    ⋮                                    ⋱                        ⋮
 2060.87      4131.7   6138.57  8196.79     8.38448f6  8.38655f6  8.3886f6
 2061.77      4132.59  6139.38  8197.22    

Using the high-level GPU array functionality made it easy to perform this computation on the GPU. However, we didn't learn about what's going on under the hood, and that's the main goal of this workshop. It's time to write our own kernels!

In [19]:
function vadd!(C, A, B)
    for i in 1:length(A)
        @inbounds C[i] = A[i] + B[i]
    end
    return nothing
end

vadd! (generic function with 1 method)

In [20]:
A = ones(10)
B = ones(10)
C = similar(B)
vadd!(C, A, B)
C

10-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

In [21]:
if CUDA.functional()
    # We can already run this on the GPU with the @cuda macro,
    # which will compile vadd!() into a GPU kernel and launch it
    A_d = CuArray(A)
    B_d = CuArray(B)
    C_d = similar(B_d)
    @cuda vadd!(C_d, A_d, B_d)
    C_d
end

10-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0
 2.0

The macros for the other GPU backends are `@roc`, `@oneapi` and `@metal`.

The performance are just terrible because each thread on the GPU would be performing the same loop! So we have to remove the loop over all elements and instead use the special `threadIdx` and `blockDim` functions, analogous respectively to `threadid` and `nthreads` for multithreading.

We can split work between the GPU threads by using a special function which returns the index of the GPU thread which executes it.

In [22]:
function vadd2!(C, A, B)
    index = threadIdx().x   # linear indexing, so only use `x`
    @inbounds C[index] = A[index] + B[index]
    return nothing
end

vadd2! (generic function with 1 method)

In [23]:
if CUDA.functional()
    N = 2^8
    A = 2 * CUDA.ones(N)
    B = 3 * CUDA.ones(N)
    C = similar(B)

    nthreads = N
    @cuda threads=nthreads vadd2!(C, A, B)
end

CUDA.HostKernel{typeof(vadd2!), Tuple{CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}}}(vadd2!, CuFunction(Ptr{CUDA.CUfunc_st} @0x000000003ad87e00, CuModule(Ptr{CUDA.CUmod_st} @0x000000003d8c3a50, CuContext(0x0000000006d0c9f0, instance 2795a03b73c6d58a))), CUDA.KernelState(Ptr{Nothing} @0x00007f70a7200000))

In [24]:
if CUDA.functional()
    all(Array(C) .== 5.0)
end

true

The syntax is similar for the other GPU backends!
```julia
groupsize = length(A)
@roc groupsize=groupsize vadd!(C, A, B)

items = length(A)
@oneapi items=items vadd!(C, A, B)

nthreads = length(A)
@metal threads=nthreads vadd!(C, A, B)
```

To do even better, we need to parallelize more. GPUs have a limited number of threads they can run on a single streaming multiprocessor (SM), but they also have multiple SMs. To take advantage of them all, we need to run a kernel with multiple blocks. We'll divide up the work like this:

![gpu_threads_block](./Graphics/gpu_threads_block.png)

This diagram was borrowed from a description of the NVIDIA C/C++ library; in Julia, threads and blocks begin numbering with 1 instead of 0. In this diagram, the 4096 blocks of 256 threads (making 1048576 = 2^20 threads) ensures that each thread increments just a single entry; however, to ensure that arrays of arbitrary size can be handled, let's still use a loop:

In [25]:
function vadd3!(C, A, B)
    index = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    stride = gridDim().x * blockDim().x
    for i = index:stride:length(B)
        @inbounds C[index] = A[index] + B[index]
    end
end

vadd3! (generic function with 1 method)

In [26]:
if CUDA.functional()
    nthreads = CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)
end

1024

The maximum number of allowed threads to launch depends on your GPU!

In [27]:
if CUDA.functional()
    N = 2^14
    A = 2 * CUDA.ones(N)
    B = 3 * CUDA.ones(N)
    C = similar(B)

    # smallest integer larger than or equal to N / nthreads
    numblocks = ceil(Int, N/nthreads)
end

16

In [28]:
if CUDA.functional()
    @cuda threads=nthreads blocks=numblocks vadd3!(C, A, B)
end

CUDA.HostKernel{typeof(vadd3!), Tuple{CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}}}(vadd3!, CuFunction(Ptr{CUDA.CUfunc_st} @0x000000003a981750, CuModule(Ptr{CUDA.CUmod_st} @0x000000003980b560, CuContext(0x0000000006d0c9f0, instance 2795a03b73c6d58a))), CUDA.KernelState(Ptr{Nothing} @0x00007f70a7200000))

In [29]:
 all(Array(C) .== 5.0)

true

CUDA.jl supports indexing in up to 3 dimensions (x, y and z, e.g. `threadIdx().z`). This is convenient for multidimensional data where thread blocks can be organised into 1D, 2D or 3D arrays of threads.

To automatically select an appropriate number of threads, it is recommended to use the launch configuration API. This API takes a compiled (but not launched) kernel, returns a tuple with an upper bound on the number of threads, and the minimum number of blocks that are required to fully saturate the GPU:

To optimize the number of threads, we can first create the kernel without launching it, query it for the number of threads supported, and then launch the compiled kernel:

In [32]:
# compile kernel
kernel = @cuda launch=false vadd3!(C, A, B)

# extract configuration via occupancy API
config = launch_configuration(kernel.fun)

# number of threads should not exceed size of array
threads = min(length(A), config.threads)
print(threads,'\n')

# smallest integer larger than or equal to length(A)/threads
blocks = cld(length(A), threads)
print(blocks,'\n')

# launch kernel with specific configuration
kernel(C, A, B; threads, blocks)

1024
16

**Debugging**: Many things can go wrong with GPU kernel programming and unfortunately error messages are sometimes not very useful because of how the GPU compiler works.

Conventional print-debugging is often a reasonably effective way to debug GPU code. CUDA.jl provides macros that facilitate this:
- `@cushow` (like @show): visualize an expression and its result, and return that value.
- `@cuprintln` (like println): to print text and values.
- `@cuaassert` (like @assert) can also be useful to find issues and abort execution.

GPU code introspection macros also exist, like `@device_code_warntype`, to track down type instabilities.

In [33]:
function gpu_add_print!(y, x)
    index = threadIdx().x    # this example only requires linear indexing, so just use `x`
    stride = blockDim().x
    @cuprintln("thread $index, block $stride")
    for i = index:stride:length(y)
        @inbounds y[i] += x[i]
    end
    return nothing
end

if CUDA.functional()
    x_d = CUDA.rand(10)
    y_d = CUDA.rand(10)
    @cuda threads=10 gpu_add_print!(y_d, x_d)
    synchronize()
end

thread 1, block 10
thread 2, block 10
thread 3, block 10
thread 4, block 10
thread 5, block 10
thread 6, block 10
thread 7, block 10
thread 8, block 10
thread 9, block 10
thread 10, block 10


**Conclusion**: Keep in mind that the high-level functionality of CUDA often means that you don't need to worry about writing kernels at such a low level. However, there are many cases where computations can be optimized using clever low-level manipulations. The kernels implemented in Julia give you all the flexibility and performance a GPU has to offer, within a familiar language.

A typical approach for porting or developing an application for the GPU is as follows:
- develop an application using generic array functionality, and test it on the CPU with the `Array` type;
- port your application to the GPU by switching to the `CuArray` type;
- disallow the CPU fallback ("scalar indexing") to find operations that are not implemented for or incompatible with GPU execution;
- (optional) use lower-level, CUDA-specific interfaces to implement missing functionality or optimize performance.   

**Exercise**: GPU-port the `sqrt_sum` function we saw in te first notebook:

In [ ]:
function sqrt_sum(A)
    T = eltype(A)
    s = zero(T)
    for i in eachindex(A)
        @inbounds s += sqrt(A[i])
    end
    return s
end

# References:
- https://enccs.github.io/Julia-for-HPC/GPU/
- https://cuda.juliagpu.org/stable/
- https://www.youtube.com/watch?v=Fz-ogmASMAE
- https://www.cherryservers.com/blog/gpu-vs-cpu-what-are-the-key-differences
- https://developer.nvidia.com/blog/tag/cuda-refresher/
- https://i.redd.it/yr9h5cpyzpn21.jpg
- https://docs.nvidia.com/cuda/
- https://www.youtube.com/watch?v=Hz9IMJuW5hU
- https://julialang.org/learning/